# Creating multi-mission, multi-temporal datacube

In [2]:
import openeo

This notebooks shows how to combine timeseries data from two popular missions, Sentinel-1 and Sentinel-2 in a single datacube for further processing. It can be considered a basic template for many use cases.

The example uses precomputed backscatter, but it is also possible to compute backscatter on the fly, which works globally, but also consumes more credits. Hence for a project with fixed test areas, it is somewhat easier/cheaper to precompute data.

The option for computing backscatter on the fly can be enabled by using the "SENTINEL1_GRD" collection, and also requires the use of an extra process.

We also create 10-daily composites, and apply linear interpolation to avoid gaps. Specific methods may of course require different cloud masking and preprocessing options.

In [3]:


c=openeo.connect("openeo.cloud")
c.authenticate_oidc()

sentinel2 = c.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["B02", "B03", "B04","SCL"],
    max_cloud_cover=75
)

sentinel2 = sentinel2.process(
            "mask_scl_dilation",
            data=sentinel2,
            scl_band_name="SCL",
            kernel1_size=17, kernel2_size=77,
            mask1_values=[2, 4, 5, 6, 7],
            mask2_values=[3, 8, 9, 10, 11],
            erosion_kernel_size=3)

sentinel2 = sentinel2.aggregate_temporal_period("dekad",reducer="median")\
    .apply_dimension(dimension="t", process="array_interpolate_linear")



To authenticate: visit https://aai.egi.eu/device?user_code=JRZW-KOXP .
Authorized successfully.
Authenticated using device code flow.


/home/driesj/python/openeo-python-client/openeo/rest/connection.py:1013: UserWarning: SENTINEL2_L2A property filtering with properties that are undefined in the collection metadata (summaries): eo:cloud_cover.
  return DataCube.load_collection(


In [ ]:

S1_collection = "SENTINEL1_GRD_SIGMA0"
#S1_collection = "SENTINEL1_GRD"

sentinel1 = c.load_collection(
    S1_collection,
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["VV","VH"]
)

if S1_collection == "SENTINEL1_GRD":
    S1bands = S1bands.sar_backscatter(
        coefficient='sigma0-ellipsoid',
        local_incidence_angle=False,
        elevation_model='COPERNICUS30')

sentinel1 = sentinel1.aggregate_temporal_period("dekad",reducer="median")\
    .apply_dimension(dimension="t", process="array_interpolate_linear")

Now we can simply combine both cubes. Resampling is performed implicitly if needed, but explicit resampling can also be specified.

In [ ]:
merged = sentinel2.merge_cubes(sentinel1)

The next block receives the combined Sentinel-1 and Sentinel-2 input, and transforms it using whatever method. 
This can be for instance a neural network based on PyTorch.

This example uses blocks of 128x128 pixels, with an 8 pixel overlap. Sizes for the time and bands dimensions are not specified, which means they will be fully included.

The UDF in this example also shows how to print statements to the logging, this is an easy way to get a better sense of the XArray data that is passed in.

In [4]:
my_udf = openeo.UDF("""
from openeo.udf import XarrayDataCube
from openeo.udf.debug import inspect

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    inspect(array,level="ERROR",message="inspecting input cube")
    array.values = 0.0001 * array.values
    return cube
""")

fused = merged.apply_neighborhood(my_udf, size=[
        {'dimension': 'x', 'value': 112, 'unit': 'px'},
        {'dimension': 'y', 'value': 112, 'unit': 'px'}
    ], overlap=[
        {'dimension': 'x', 'value': 8, 'unit': 'px'},
        {'dimension': 'y', 'value': 8, 'unit': 'px'}
    ])

In [5]:
spatial_extent = {'west': 4.45, 'east': 4.70, 'south': 51.16, 'north': 51.22, 'crs': 'epsg:4326'}
job=fused.filter_bbox(spatial_extent).execute_batch()

0:00:00 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': send 'start'
0:00:27 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:00:33 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:00:40 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:00:48 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:00:58 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:01:12 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:01:28 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:01:49 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': queued (progress N/A)
0:02:13 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': running (progress N/A)
0:02:49 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': running (progress N/A)
0:03:27 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': running (progress N/A)
0:04:14 Job 'vito-j-9f8722a36c244e96a6497a3d6eb4a267': running (progress N/A)
0:05

When the job is finished, results can be downloaded, this is explained in the documentation:
https://open-eo.github.io/openeo-python-client/batch_jobs.html#download-batch-job-results
Here we show how to get the logs, with the output from the inspect process:

In [6]:
job.logs()

[{'id': '[1678997260604, 684092788]',
  'time': '2023-03-16T20:07:40.604Z',
  'level': 'info',
  'message': "EJR creating job_id='j-9f8722a36c244e96a6497a3d6eb4a267' created='2023-03-16T20:07:40Z'"},
 {'id': '[1678997260604, 684093230]',
  'time': '2023-03-16T20:07:40.604Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs`: start 2023-03-16 20:07:40.604926'},
 {'id': '[1678997260605, 684093645]',
  'time': '2023-03-16T20:07:40.605Z',
  'level': 'debug',
  'message': "Doing EJR request `POST https://jobregistry.openeo.vito.be/jobs` headers.keys()=dict_keys(['Authorization'])"},
 {'id': '[1678997260679, 684094110]',
  'time': '2023-03-16T20:07:40.679Z',
  'level': 'info',
  'message': 'EJR response on `POST /jobs`: 201'},
 {'id': '[1678997260679, 684094499]',
  'time': '2023-03-16T20:07:40.679Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs`: end 2023-03-16 20:07:40.679666, elapsed 0:00:00.074740'},
 {'id': '[1678997262123, 684107042]',
  'time': '2023-03-16T20:07:42.123Z',
  'level': 'info',
  'message': "Starting job 'j-9f8722a36c244e96a6497a3d6eb4a267' from user User('dfa678cb9ab17f65d4f025e30fac5e0d90116176e44fd17d703419322747cbbd@egi.eu', {'oidc_userinfo': {'sub': 'dfa678cb9ab17f65d4f025e30fac5e0d90116176e44fd17d703419322747cbbd@egi.eu', 'voperson_verified_email': ['jeroen.dries@vito.be'], 'email_verified': True, 'eduperson_assurance': ['https://refeds.org/assurance/IAP/low', 'https://aai.egi.eu/LoA#Substantial'], 'eduperson_entitlement': ['urn:mace:egi.eu:group:vo.openeo.cloud:role=early_adopter#aai.egi.eu', 'urn:mace:egi.eu:group:vo.openeo.cloud:role=employee#aai.egi.eu', 'urn:mace:egi.eu:group:vo.openeo.cloud:role=member#aai.egi.eu', 'urn:mace:egi.eu:group:vo.openeo.cloud:role=vm_operator#aai.egi.eu', 'urn:mace:egi.eu:group:vo.openeo.cloud:admins:role=member#aai.egi.eu', 'urn:mace:egi.eu:group:vo.openeo.cloud:admins:role=owner#aai.egi.eu'], 'email': 'jeroen.dries@vito.be'}}) (proxy user 'driesj')"},
 {'id': '[1678997262164, 684109048]',
  'time': '2023-03-16T20:07:42.164Z',
  'level': 'info',
  'message': "EJR update job_id='j-9f8722a36c244e96a6497a3d6eb4a267' data={'proxy_user': 'driesj'}"},
 {'id': '[1678997262165, 684109488]',
  'time': '2023-03-16T20:07:42.165Z',
  'level': 'debug',
  'message': 'EJR Request `PATCH /jobs/j-9f8722a36c244e96a6497a3d6eb4a267`: start 2023-03-16 20:07:42.165063'},
 {'id': '[1678997262165, 684109939]',
  'time': '2023-03-16T20:07:42.165Z',
  'level': 'debug',
  'message': "Doing EJR request `PATCH https://jobregistry.openeo.vito.be/jobs/j-9f8722a36c244e96a6497a3d6eb4a267` headers.keys()=dict_keys(['Authorization'])"},
 {'id': '[1678997264740, 684110440]',
  'time': '2023-03-16T20:07:44.740Z',
  'level': 'info',
  'message': 'EJR response on `PATCH /jobs/j-9f8722a36c244e96a6497a3d6eb4a267`: 404'},
 {'id': '[1678997264740, 684110865]',
  'time': '2023-03-16T20:07:44.740Z',
  'level': 'debug',
  'message': 'EJR Request `PATCH /jobs/j-9f8722a36c244e96a6497a3d6eb4a267`: fail 2023-03-16 20:07:44.740522, elapsed 0:00:02.575459'},
 {'id': '[1678997264827, 684117480]',
  'time': '2023-03-16T20:07:44.827Z',
  'level': 'debug',
  'message': 'job_options: {}'},
 {'id': '[1678997267922, 684143621]',
  'time': '2023-03-16T20:07:47.922Z',
  'level': 'info',
  'message': "Dry run extracted these source constraints: [(('load_collection', ('SENTINEL2_L2A', (('eo:cloud_cover', (('lte', 75),)),))), {'temporal_extent': ('2022-06-04', '2022-08-04'), 'spatial_extent': {'west': 4.45, 'south': 51.16, 'east': 4.7, 'north': 51.22, 'crs': 'epsg:4326'}, 'bands': ['B02', 'B03', 'B04', 'SCL'], 'process_type': [<ProcessType.GLOBAL_TIME: 4>], 'custom_cloud_mask': {'method': 'mask_scl_dilation', 'erosion_kernel_size': 3, 'kernel1_size': 17, 'kernel2_size': 77, 'mask1_values': [2, 4, 5, 6, 7], 'mask2_values': [3, 8, 9, 10, 11], 'scl_band_name': 'SCL'}, 'properties': {'eo:cloud_cover': {'process_graph': {'lte1': {'process_id': 'lte', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 75}, 'result': T